# TO DO:

1. Dataset Loading
Implements the pipeline to load a dataset from the src domain. Since the challenge repository doesn’t allow storing large files, loaders must download datasets from external sources into the datasets/ folder.
This pipeline is provided for several graph-based datasets. For any other src domain, participants are allowed to transform graph datasets into the corresponding domain through our provided lifting mappings –or just dropping their connectivity to get point-clouds.
(Bonus) Designing a loader for a new dataset (ones that are not already provided in the tutorials) will be positively taken into consideration in the final evaluation.

2. Pre-processing the Dataset
Applies the lifting transform to the dataset.
Needs to be done through the PreProcessor, which we provide in
modules/io/preprocess/preprocessor.py.

3. Running a Model over the Lifted Dataset 
Creates a Neural Network model that operates over the dst domain, leveraging TopoModelX for higher order topologies or torch_geometric for graphs.
Runs the model on the lifted dataset.